In [1]:
import uuid
import os
import asyncio
from pprint import pp
from dotenv import load_dotenv
from tempfile import mkdtemp

from jobs import Worker
from shared import MongoDbConnector, save_uploaded_file, upload_blob

load_dotenv('../assets/.env_dev')

MONGO_URI = os.getenv('MONGO_URI')
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
BUCKET_NAME = os.getenv('BUCKET_NAME')
DB_NAME = os.getenv('DB_NAME')

db_connector = MongoDbConnector(connection_uri=MONGO_URI, database_id=DB_NAME, connector_id="test_worker")

In [3]:
db_connector.pending_jobs()

[{'_id': ObjectId('66a572d06bbfeecd576539de'),
  'status': 'PENDING',
  'job_id': 'b863a655-5d8c-4de3-a6cd-750b0cb947a2',
  'omex_path': 'uploads/b863a655-5d8c-4de3-a6cd-750b0cb947a2/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'comparison_id': 'test0',
  'timestamp': '2024-07-27 22:21:03.274290',
  'ground_truth_report_path': None,
  'include_outputs': True}]

In [2]:
omex_dir = "../model-examples/sbml-core/Elowitz-Nature-2000-Repressilator"
uploaded_file = omex_dir + '.omex'
ground_truth_report = None  # os.path.join(omex_dir, 'reports.h5')

os.path.exists(uploaded_file)

True

In [3]:
job_id = str(uuid.uuid4())
_time = db_connector.timestamp()

In [ ]:
pending_jobs = db_connector.pending_jobs()
if len(pending_jobs) > 0:
    print(f'There are {len(pending_jobs)} pending jobs that will be deleted.')
    for doc in pending_jobs:
        db_connector.db.pending_jobs.delete_one(doc)

In [3]:
db_connector.pending_jobs()

[{'_id': ObjectId('66a572d06bbfeecd576539de'),
  'status': 'PENDING',
  'job_id': 'b863a655-5d8c-4de3-a6cd-750b0cb947a2',
  'omex_path': 'uploads/b863a655-5d8c-4de3-a6cd-750b0cb947a2/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'comparison_id': 'test0',
  'timestamp': '2024-07-27 22:21:03.274290',
  'ground_truth_report_path': None,
  'include_outputs': True}]

In [4]:
db_connector.in_progress_jobs()

[{'_id': ObjectId('66a5753129da901ff6c095de'),
  'job_id': 'ebdd3c3c-ffe3-46db-b569-0ebd83fbe4a1',
  'status': 'IN_PROGRESS',
  'timestamp': '2024-07-27 22:31:13.731893',
  'comparison_id': 'test0',
  'worker_id': '6dc08aba-ad2b-4ff2-bf22-5b288f1a8287'}]

In [7]:
db_connector.completed_jobs()

[{'_id': ObjectId('66a575b5f6349a186cc5b38f'),
  'job_id': '49c56320-6950-44bc-892f-a1718a4c7cd2',
  'status': 'COMPLETED',
  'timestamp': '2024-07-27 22:33:25.368101',
  'comparison_id': 'test0',
  'results': {'results': [{'species_name': 'LacI protein',
     'mse': {'amici': {'amici': 0.0,
       'copasi': 3528884.807740261,
       'tellurium': 0.0005853579775907033},
      'copasi': {'amici': 3528884.807740261,
       'copasi': 0.0,
       'tellurium': 3528864.647965566},
      'tellurium': {'amici': 0.0005853579775907033,
       'copasi': 3528864.647965566,
       'tellurium': 0.0}},
     'proximity': {'amici': {'amici': True,
       'copasi': False,
       'tellurium': True},
      'copasi': {'amici': False, 'copasi': True, 'tellurium': False},
      'tellurium': {'amici': True, 'copasi': False, 'tellurium': True}},
     'output_data': {'amici': [1971.2685726016928,
       2056.478842813443,
       2133.4749996610312,
       2200.7609967528674,
       2256.962432237415,
       230

In [ ]:
# save_dest = mkdtemp()
# bucket params
upload_prefix = f"uploads/{job_id}/"
bucket_prefix = f"gs://{BUCKET_NAME}/" + upload_prefix

# Save uploaded omex file to Google Cloud Storage
# omex_fp = await save_uploaded_file(uploaded_file, save_dest)
# for local dev, the file is already written, duh!
uploaded_file_name = uploaded_file.split('/')[-1]
omex_fp = uploaded_file
omex_blob_dest = upload_prefix + uploaded_file_name
omex_path = omex_blob_dest  # bucket_prefix + uploaded_file

upload_blob(bucket_name=BUCKET_NAME, source_file_name=omex_fp, destination_blob_name=omex_blob_dest)

In [ ]:
# Save uploaded reports file to Google Cloud Storage if applicable
report_fp = None
report_blob_dest = None
if ground_truth_report:
    # report_fp = await save_uploaded_file(ground_truth_report, save_dest)
    report_blob_dest = upload_prefix + ground_truth_report
    upload_blob(BUCKET_NAME, report_fp, report_blob_dest)
report_path = report_blob_dest if report_fp else None # bucket_prefix + ground_truth_report.filename if report_fp 

In [ ]:
# run insert job
simulators = ['amici', 'copasi', 'tellurium']
comparison_id = f"test_{job_id}"

In [ ]:
pending_job_doc = await db_connector.insert_job_async(
        collection_name="pending_jobs",
        status="PENDING",
        job_id=job_id,
        omex_path=omex_path,
        simulators=simulators,
        comparison_id=comparison_id or f"uniform-time-course-comparison-{job_id}",
        timestamp=_time,
        ground_truth_report_path=report_path,
        include_outputs=True
)

In [9]:
compare = db_connector.in_progress_jobs().copy().pop()['comparison_id']

db_connector.fetch_job(compare)

{'_id': ObjectId('66a575b5f6349a186cc5b38f'),
 'job_id': '49c56320-6950-44bc-892f-a1718a4c7cd2',
 'status': 'COMPLETED',
 'timestamp': '2024-07-27 22:33:25.368101',
 'comparison_id': 'test0',
 'results': {'results': [{'species_name': 'LacI protein',
    'mse': {'amici': {'amici': 0.0,
      'copasi': 3528884.807740261,
      'tellurium': 0.0005853579775907033},
     'copasi': {'amici': 3528884.807740261,
      'copasi': 0.0,
      'tellurium': 3528864.647965566},
     'tellurium': {'amici': 0.0005853579775907033,
      'copasi': 3528864.647965566,
      'tellurium': 0.0}},
    'proximity': {'amici': {'amici': True, 'copasi': False, 'tellurium': True},
     'copasi': {'amici': False, 'copasi': True, 'tellurium': False},
     'tellurium': {'amici': True, 'copasi': False, 'tellurium': True}},
    'output_data': {'amici': [1971.2685726016928,
      2056.478842813443,
      2133.4749996610312,
      2200.7609967528674,
      2256.962432237415,
      2300.8927481621536,
      2331.6203242747

In [ ]:
jobs = db_connector.pending_jobs().copy()

jobs

In [ ]:
job = jobs.pop()

job

In [4]:
from jobs import BUCKET_NAME

BUCKET_NAME

'bio-check-requests-1'

In [5]:
from jobs import Supervisor

supervisor = Supervisor(db_connector=db_connector)

In [ ]:
result = await supervisor.check_jobs()

In [ ]:
result

In [ ]:
worker = Worker(job_params=job)

pp(f'Job result of worker: {worker.job_result}')
# request specific params

In [ ]:
pp(worker.job_result)

In [8]:
db_connector.completed_jobs().pop()['comparison_id']

'test0'

In [9]:
pending = supervisor.db_connector.pending_jobs()
in_progress = supervisor.db_connector.in_progress_jobs()
completed = supervisor.db_connector.completed_jobs()

exists_in_progress = supervisor.exists_in_next_queue(current_queue=pending, next_queue=in_progress)

exists_in_progress

False

In [11]:
supervisor.db_connector.pending_jobs()

[{'_id': ObjectId('66a572d06bbfeecd576539de'),
  'status': 'PENDING',
  'job_id': 'b863a655-5d8c-4de3-a6cd-750b0cb947a2',
  'omex_path': 'uploads/b863a655-5d8c-4de3-a6cd-750b0cb947a2/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'comparison_id': 'test0',
  'timestamp': '2024-07-27 22:21:03.274290',
  'ground_truth_report_path': None,
  'include_outputs': True}]

In [ ]:
from bio_check.service import Service

service = Service()

In [15]:
supervisor._job_exists(comparison_id='test0', collection_name='in_progress_jobs')

True

In [ ]:
from shared import download_blob

source_blob = job['omex_path']
out_dir = mkdtemp()
local_omex_fp = os.path.join(out_dir, source_blob.split('/')[-1])

local_omex_fp

In [ ]:
source_blob

In [ ]:
download_blob(bucket_name=BUCKET_NAME, source_blob_name=source_blob, destination_file_name=local_omex_fp)

In [ ]:
os.listdir(out_dir)

In [ ]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(source_blob.replace('gs://bio-check-requests-1/', ''))

In [ ]:
dir(blob)

In [ ]:
blob.download_to_filename(local_omex_fp) 